# 스파크 스트리밍 실습 2교시

> 구조화된 API 를 통해 대용량 데이터 처리를 배웠으나, finite volumes 데이터에 대한 처리였고, 때때로 리얼타임 수준의 연속적인 데이터에 대한 처리가 필요한 경우가 있는데, 이러한 경우에도 마찬가지로 Structured APIs를 통해 어떻게 처리가 가능한 지 학습합니다

## 학습 과제
* '구조화된 스트리밍'의 핵심 철학과 입력 데이터 스트림을 '언바운디드 테이블'로 생각하고 처리하는 데이터 모델을 학습합니다
* '스트리밍 질의문'을 정의, 시작, 재시작 그리고 모니터링 하는 기법을 학습합니다
* 다양한 내장 스트리밍 소스와 싱크 그리고 커스텀 스트리밍 싱크 들에 대해 학습합니다
* 스트리밍 집계 혹은 스트림-스트림 조인 등의 매니지드 스테이트풀 연산 들을 튜닝하는 방법을 학습합니다
* 커스텀 스테이트풀 계산 표현 기술을 학습합니다
* 실제 예제는 [Learning Spark v2@Github](https://github.com/databricks/LearningSparkV2)을 참고하였습니다 

## 1. 아파치 스파크 스트림 처리 엔진의 진화 - Evolution of the Apache Spark Stream Processing Engine

![figure.8-1](images/figure.8-1.png)

* 스트리밍 처리는 끝없는 데이터 스트림의 연속적인 처리의 과정을 말합니다.
  - 전통적인 실시간 데이터 처리 모델은 *record-at-a-time processing model* 즉, 한 번에 하나의 레코드를 처리하는 방식입니다
  - 레코드 단위로 수행되기 때문에 낮은 레이턴시를 가지며, 상대적으로 느린 노드들(straggler nodes) 혹은 실패한 노드들에 의한 복원력이 떨어집니다

### 1-1. 마이크로 배치 스트림 처리의 출현 - The Advent of Micro-Batch Stream Processing

![figure.8-2](images/figure.8-2.png)

* 레코드 단위의 작업이 아니라, 아주 작은 연속적인 Map/Reduce 스타일의 배치 작업을 (일명 마이크로-배치 작업) 통해 연속적인 작은 스트림 데이터 덩어리들을 처리합니다
  - 스파크 스트리밍 작업은 1초 단위의 마이크로-배치 작업을 통해 입력 스트림이 나누어지고 수행됩니다
  - 개별 작업은 deterministic task 로 구분되고, 노드에 무관하게 배치작업이 할당되고 처리될 수 있습니다
* 마이크로 배치 모델의 장점
  - 실패한 작업의 시작과 끝을 명확하게 알 수 있기 때문에, straggler executors 혹은 failures 노드에 대한 작업을 효과적으로 다시 수행에 용이합니다
  - 작업의 멱등성 덕분에, 동일한 반복적인 작업을 수행하더라도 exactly-once 수행이 가능하도록 처리할 수 있습니다
* 마이크로 배치 모델의 단점
  - 1초 단위의 배치 작업으로 구성되므로 millisecond-level 레이턴시 제공은 어렵습니다
  - 다만, 밀리 초 단위로 스트리밍 처리 결과를 사용하는 대시보드 혹은 서비스는 그리 흔하지 않습니다
  - 대게 아파치 카프카와 같은 메시지 큐 서비스와 연동되기 마련인데, 이러한 인프라의 경우 처리량을 높이기 위해 버퍼링 혹은 초 단위 이상의 블록단위의 작업이 이루어지는 경우가 많습니다

### 1-2. 스파크 스트리밍 (DStreams)로 부터 얻은 교훈 - Lessons Learned from Spark Streaming (DStreams)

* 배치와 스트림 처리의 단일 API 부재
  - 동일한 작업의 경우에도 배치와 스트림 처리에 반복적인 코드 작성이 필요함
* 논리 및 물리적 플랜간의 분리 부재
  - DStreams 연산의 경우 개발자가 정의한 물리적인 계획대로만 수행되며 논리적인 최적화 과정이 없음
* 이벤트 타임 윈도우 지원의 부재
  - DStreams 경우 이벤트 타임 기준의 윈도우 워터마크 처리를 지원하지 않아 이벤트타임 기준 작업은 별도로 구현이 필요하여 어려움이 있음


### 1-3. 구조화된 스트리밍의 철학 - The Philosophy of Structured Streaming

* 배치와 스트림 처리에 대해 단일화된 통합 프로그래밍 모델과 인터페이스를 제공
  - 내결함성, 최적화 및 지연 데이터 등의 복잡한 처리를 SQL 혹은 데이터프레임 배치 처리와 유사하게 처리
* 더 넓어진 스트리밍 애플리케이션의 응용범위
  - 배치와 실시간 처리에서 대용량 데이터 처리는 점점 복잡해져 왔다
  - 구조화된 스트리밍은 전통적인 스트림 처리보다 적용가능성이 더 넓어졌다
    - 시간 단위로 실행되는 주기적인 작업들부터 전통적인 스트리밍 애플리케이션에 이르기까지

## 2. 구조화된 스트리밍 프로그래밍 모델 - The Programming Model of Structured Streaming

![figure.8-3](images/figure.8-3.png)

> '배치 애플리케이션'에서 친숙하고 다루기 용이한 "테이블" 개념을 '구조화된 스트리밍'에서는 그 개념을 확장시켜, `지속적으로 추가 가능하여, 무제한의 데이터가 저장되는 스트림 테이블`으로 생각되는 테이블이라고 정의합니다.

![figure.8-4](images/figure.8-4.png)

* Unbounded 입력 테이블에 새로운 로우가 들어올 때마다 구조화된 스트리밍 처리는 이루어지지 않지만, 결과적으로 임의의 시간 T 까지 수신되는 모든 데이터에 대한 처리를 배치 처리와 동일하게 처리할 수 있게 합니다
  - 마치 정적인 테이블에 질의문을 수행하는 것과 유사하게 '구조화된 스트리밍'은 입력 질의문을 '스트리밍 실행 계획'으로 자동 변환을 수행하며 이를 `incrementalization` 이라고 말합니다.
    - 그림에서의 트리거링의 시간 별로 새로운 데이터를 확인하고, 점진적으로(incrementally) 결과를 업데이트 하기 때문에 '증분화'라고 표현하는 것 같습니다
    - 예를 들어 초 당 평균 100개의 레코드가 발생하는 경우 특정 카테고리 기준으로 집계연산 (Count 등)을 수행한 결과를 저장하는 예제
  - 매 트리거 마다, 증분된 입력 데이터에 대한 질의문을 처리한 결과를 결과 테이블에 저장하고, 이전까지 처리된 로우들을 싱크로 보내는 작업을 수행합니다
* 입력 테이블, 증분 질의문, 결과 테이블까지의 처리가 '구조화된 스트리밍' 처리에서 수행되는 작업이라고 이해할 수 있습니다
  - 가상의 소스, 싱크 테이블을 유지 관리하는 기능
  - 구조화된 API 등을 통해 증분 쿼리를 수행하는 기능
  - 다양한 데이터 소스, 싱크 테이블을 읽고 쓸 수 있는 기능

#### 다양한 출력 모드
  - *Append mode*
    - 신규로 발생한 로우의 추가만 가능한 모드이며, 기존 결과 테이블에 저장된 로우들은 변경되지 않습니다
  - *Update mode*
    - 마지막 트리거 이후에 발생한 모든 로우가 외부 저장소에 변경되는 모드이며, MySQL 과 같이 업데이트를 지원하는 싱크 저장소만 사용할 수 있습니다
  - *Complete mode*
    - 매번 결과 테이블 전체를 갱신하는 모드이며, 모든 로우를 매번 출력할 만큼 충분히 결과가 작은 경우에 사용할 수 있습니다

#### Fully Materialized ?
  - '구조화된 스트리밍' 처리에 있어서 'complete mode'를 제외하고 전체 '완전한 구체화'되지 않습니다
  - 다른 모드에서는, 변경 사항에 대해 어떻게 구체화 할 지에 대한 정보를 ("state"라고 알려진) 통해 향후 계산 및 갱신할 수 있도록 유지하게 됩니다

## 3. 구조화된 스트리밍 질의문의 기본 - The Fundamentals of a Structured Streaming Query

### 3-1. 스트리밍 질의문을 정의하는 다섯 단계 - Five Steps to Define a Streaming Query
> spark 객체를 통해서 데이터를 읽어오는 reader 를 만들고 데이터 카운트를 세는 counter 그리고 결과를 저장할 writer 를 통해 모든 작업이 수행됩니다
<br>
> 0:spark -> 1:reader -> 2:counter -> 3:writer -> 4:trigger -> 5:query 순서의 파이프라인을 생성합니다

#### Step 1: 입력 소스 테이블 정의하기 - { spark, reader }
  - 스파크 세션을 생성하는 과정을 동일하며, spark.read 대신 readStream 을 통해 데이터프레임을 생성하는 점이 다릅니다
    - DataFrameReader 와 유사한 DataStreamReader 를 통해 데이터를 읽어오게 됩니다

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

In [3]:
# 소켓 커넥션을 통해 수신된 스트림 데이터를 통해 데이터 프레임을 생성하는 예제입니다
reader = (
    spark
    .readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .load()
)
# 여기서 lines 는 localhost:9999 소켓으로부터 여러줄의 텍스트 데이터를 가진 unbounded table 을 정의합니다 (not running but declare)
# DataFrameReader 는 text 뿐만 아니라 Parquet, ORC, JSON 등 다양한 포맷까지 기본적으로 지원합니다

#### Step 2: 데이터 가공하기 - { counter }

> 이제부터는 라인을 단어로 구분하거나, 횟수를 세는 등의 일반적인 데이터프레임 연산을 수행합니다

In [4]:
# 공백을 " "으로 명시해도 되지만, \t, space 등을 정규식으로 인식하도록 \s 가 더 유용합니다
counter = reader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("word").count().alias("count")

> 여기까지는 실행되는 코드라기 보다는 어떻게 변환 할 지에 대한 가공에 대한 처리를 정의하는 과정이므로, show 함수 수행 시에는 스트림 데이터가 시작되지 않아 오류가 발생합니다

##### Stateless transformations
  - select, filter, map 등의 함수는 이전에 수행 되었던 로우가 다음 수행될 로우에 대해 영향이 없는 변환 작업을 스테이트리스 변환작업이라고 합니다
  - 즉, 이전 변환작업의 상태(state)가 이후 작업에 필요 없는(less) 변환(transformation) 작업입니다

##### Stateful transformations
  - 집계가 필요한 count 와 같은 함수는 이전의 누적된 카운트 상태가 필요한 변환작업을 스테이트풀 변환작업이라고 합니다
  - join, group, aggregating 과 연관된 모든 변환은 이에 해당하며, 이전 변환작업의 상태(state)가 필요한(ful) 변환 (transformation) 작업입니다
  - 대부분의 작업은 지원하지만 일부는 리소스 혹은 비용적인 면에서 수행되기 어려운 경우도 있습니다

#### Step 3: 출력 싱크 테이블 및 모드 선택하기 - { writer }
> 여기까지 데이터 변환에 대한 정의를 했다면, 변환된 데이터 저장을 위한 싱크를 작성해야 합니다. 

* DataFrame.write 대신 DataFrame.writeStream 함수를 통해 DataStreamWriter 생성을 통해 생성된 데이터프레임을 저장합니다

In [5]:
# 아래와 같이 괄호()를 통해 묶는 경우 줄바꿈을 하더라도 backslash(\)를 통해 구분하지 않아도 되어 코드 가독성이 높아집니다
writer = (
    counter
    .writeStream
    .format("console") # 결과를 콘솔에 출력합니다
    .outputMode("complete") # 매번 전체 데이터를 내보냅니다
)

##### 출력 모드 별 설계 방향
* Append mode
  - 기본 (default) 출력 모드이며, 이전 데이터가 이후의 데이터에 영향을 주지 않도록 변환처리를 설계해야만 합니다.
  - 데이터의 추가만 가능한 모드이므로, 상태가 없는 스테이트리스 한 변환을 그대로 출력하는 예제가 대표적입니다
  - 집계를 수행하는 경우, 집계에 대한 킷값이 중복되어 출력될 것이므로, 시간에 대한 추가적인 필드를 저장하건, 단순 집계를 통해 처리는 가능합니다
    - Window 즉 Buffering 을 하지 않는 상황에서는 초 단위의 집계결과만 출력할 수 밖에 없기 때문에 이러한 성질을 이해하고 있어야 합니다
* Complete mode
  - 항상 모든 출력을 그대로 내보내는 모드이기 때문에, 전체 집계 데이터가 메모리에 남아 있어도 될만큼 충분히 작도록 변환 처리를 설계해야 합니다
  - 대부분 입력 데이터를 특정 집계 함수를 통해 최종 결과 지표들만 출력하는 경우가 많습니다
    - 위의 단어 숫자를 세는 예제와 같이 입력 데이터 전체가 아니라 상위 N개의 단어만 출력하게 됩니다
* Update mode
  - 마지막 트리거 이후에 발생한 결과를 갱신하는 모드이므로 출력 저장소가 업데이트를 지원해야만 합니다

##### 파일, 카프카 및 임의의 저장소 출력
> 기본적으로 파일이나 카프카와 같이 스트리밍 저장을 지원하는 싱크도 있지만 natively 지원하지 않는 다양한 오픈소스 저장소의 경우 (batch 에서는 저장이 가능한) foreachBatch 혹은 foreach API 를 통해 저장이 가능합니다. 다만, 이러한 경우 exactly-once 를 보장하지 못합니다|

#### Step 4: 데이터 처리 상세 설계 - { trigger }
> 스트리밍 질의를 시작하기 전에 데이터 처리 실행을 위한 상세한 설계를 하는데, 얼마나 자주 혹은 어디에 저장할지 등을 결정합니다.

##### 트리거링 상세
> 새로운 데이터의 발견 및 처리를 위한 트리거를 언제 어떻게 기동할 지를 결정합니다
* Default
  - 언제 기동될 지를 명시적으로 기술하지 않는 설정이 디폴트이며, **이전에 수행된 마이크로배치 작업이 종료된 직후**에 수행됩니다
* Processing time with trigger interval
  - 명시적으로 *ProcessingTime* 트리거를 *interval*과 함께 지정하여 고정된 간격으로 마이크로배치 작업이 수행됩니다
* Once
  - 단 한 번의 마이크로배치 스트리밍 쿼리가 수행되며, 신규로 수신된 모든 데이터를 한 번의 배치작업에서 수행됩니다.
    - 외부 스케줄러 시스템을 통해서 커스텀 스케줄을 통한 수행을 해야할 필요가 있는 경우 유용합니다
* Continuous
  - Spark 3.0 버전의 실험적이 모드이며 스트리밍 쿼리가 마이크로배치가 아니라 지속적인 데이터 처리를 수행합니다
    - 기존 마이크로배치가 초 단위의 레이턴시를 제공했다면 Continuous 는 밀리초 수준의 레이턴시를 제공합니다

In [6]:
# 성공적으로 수행한 마지막 상태(state)를 저장하기 위한 지속성(persistent)있는 저장소(예: hdfs)의 위치를 지정하며, 실패한 작업의 재개 혹은 복구 시에 활용되어 exactly-once 보장.
checkpointDir="checkpoint"
trigger = (
    writer
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointDir)
)

#### Step 5: 질의문 시작하기 - { query }
> 여태까지의 모든 작업을 수행하는 단계이며 최종 trigger 객체를 start 합니다

* 시작과 동시에 checkpointDir 로 지정한 경로가 생성됩니다
* start 함수는 nonblocking 메소드이므로 즉각적으로 query 객체를 반환합니다
  - 지속적인 결과를 확인하기 위해서는 해당 query.awaitTermination() 을 통해 대기합니다
  - timeout 값을 통해 타임아웃(초) 시간을 정할 수 있으며, 지정하지 않으면 Ctrl+C(SIGINT=2)까지 수행합니다

In [7]:
query = trigger.start() # is non-blocking method
# query.awaitTermination(timeoutSeconds)

#### 5단계를 동시에 수행
> 예제를 수행하고, 해당 스트리밍 데이터 소스로부터 읽고, 처리하고, 최종 출력까지 지속적으로 수행하게 됩니다. 노트북 특성상 표준 입출력이 노트북 컨테이너의 표준출력으로 나오기 때문에 노트북 컨테이너의 로그를 통해 확인이 가능합니다

* 표준 입력으로 문자열을 입력할 소켓 서버를 생성합니다
  - 해당 도커 컨테이너에 외부 포트를 publish 하지 않았기 때문에 컨테이너에 접속하여 넷캣(nc) 도구를 이용합니다
  - nc 는 터미널 환경에서 간단히 소켓 서버를 기동할 수 있는 커맨드라인 도구입니다
| 옵션 | 설명 |
| --- | --- |
| -l (Listen mode) | 연결을 할 수 있게 계속 요청을 듣고 있게 해주는 옵션 |
| -v (Verbose) | 상세 정보를 보여주는 옵션 |
| -p (port) | Listening 포트 |

* 아래와 같이 별도의 터미널을 통해 넷캣(nc) 명령어를 실행합니다
```bash
$ docker compose exec notebook bash
# root@notebook nc -lvp 9999
Connection received on localhost 37600
hello world
data engineering intermediate course
```

* 표준 출력으로 스파크 스트리밍 실행 결과를 확인 합니다
```bash
$ docker compose logs -f notebook
21/06/27 04:27:23 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 7385 milliseconds
notebook  | -------------------------------------------
notebook  | Batch: 3
notebook  | -------------------------------------------
notebook  | +------+-----+
notebook  | |  word|count|
notebook  | +------+-----+
notebook  | |korean|    1|
notebook  | |   one|    1|
notebook  | | korea|    1|
notebook  | | hello|    1|
notebook  | |    is|    1|
notebook  | |     i|    1|
notebook  | | world|    1|
notebook  | |    am|    1|
notebook  | |number|    1|
notebook  | +------+-----+
```

In [8]:
# step0: 스파크 세션 객체를 생성합니다
from pyspark.sql import SparkSession
spark = (
    SparkSession
    .builder
    .appName("spark-streaming-5-steps")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)
# step1: 데이터를 읽어올 스트림 리더를 생성합니다
reader = (
    spark
    .readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .load()
)
# step2: 데이터로부터 단어의 수를 세는 카운터를 생성합니다
from pyspark.sql.functions import *
counter = reader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("word").count().alias("count")
# step3: 생성된 수치를 콘솔에 출력하는 출력을 생성합니다
writer = (
    counter
    .writeStream
    .queryName("wordCount") # 쿼리 테이블의 이름을 지정합니다
    .format("console") # 결과를 콘솔에 출력합니다
    .outputMode("complete") # 매번 전체 데이터를 내보냅니다
)
# step4: 얼마나 자주 수행될 지를 결정하는 트리거를 생성합니다
!rm -rf "tmp/checkpoint" # 경우에 따라서 이미 존재하는 경로의 경우 오류가 발생할 수 있기 때문에 항상 제거합니다
checkpointDir="tmp/checkpoint"
trigger = (
    writer
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointDir)
)
# step5: 해당 트리거를 시작합니다
query = trigger.start() # is non-blocking method
query.awaitTermination(60) # 노트북 특성상, 대기하면 다음 실행을 할 수 없으므로 1분간만 수행하고 종료합니다
query.stop() # 이후에 발생하는 처리는 수행되지 않으며 소켓 서버도 같이 종료됩니다

## 4. 활동 스트리밍 질의의 내부 구조 - Under the Hood of an Active Streaming Query

![figure.8-5](images/figure.8-5.png)

> 한 번 스파크 스트리밍 애플리케이션이 기동되면 아래의 내부적인 과정을 통해 작업이 수행됩니다. 여기서 반드시 기억해야 할 사항은 **'구조화된 스트리밍'은 Spark SQL 을 이용하여 수행되며, 'Spark SQL'의 최적화된 실행 엔진**을 통해 스트리밍 처리의 처리량과 레이턴시를 확보할 수 있습니다.

### 4-1. 스파크 스트리밍 실행 과정

* 1. 배치 작업과 마찬가지로 Logical Plan 을 분석 및 최적화 합니다
* 2. 백그라운드 스레드를 통해 아래의 단계를 반복 합니다
  - 트리거링 간격에 맞게 데이터 소스에 새로운 데이터가 존재하는 지 확인합니다
  - 처리해야 할 데이터가 존재한다면 '마이크로 배치' 작업을 통해 소스로 부터 새로운 데이터를 계산한 결과를 출력 모드에 따라 싱크에 저장합니다
  - 매 '마이크로 배치' 마다, 정확한 범위의 데이터가 처리되며, 결정론적(deterministically)으로 재수행 되어도 완전히 동일한 범위를 보장합니다
* 3. 쿼리가 종료되기 까지 아래와 같은 경우를 제외하면 반복은 지속됩니다
  - 쿼리 내부 오류에 의한 애플리케이션 실패 (혹은 클러스터의 장애)
  - 애플리케이션 내에서 query.stop() 메소드의 명시적인 호출에 의한 종료
  - 트리거가 Once 로 설정되어, 한 번의 '마이크로 배치' 작업이 종료된 경우


### 4-2. 'Exactly-Once' 시맨틱을 보장하는 장애복구 - Recovering from Failures with Exactly-Once Guarantees

> 한 번 실패한 '스파크 스트리밍' 작업을 재개하기 위해서는 'Spark Session'에서부터 마지막에 수행했던 과정을 그대로 수행되어야 하는데, 마지막으로 성공적으로 저장된 체크포인트 정보를 통해서, 이어서 수행될 수 있습니다.

* 반드시 동일한 '체크포인트' 경로를 지정해야 합니다
  - 해당 디렉토리는 유니크 아이덴티티 정보를 통해 쿼리의 생명주기를 결정합니다
  - 대상 디렉토리가 존재하지 않는다면, 처음부터 다시 수행되어야만 합니다
  - 체크포인트를 통해 마지막으로 성공한 바로 다음 '마이크로 배치'작업을 수행하며, 이전에 시도했던 범위와 완전히 동일한 범위의 작업을 시도하게 됩니다.


### *end-to-end exactly-once *를 보장을 위한 요건 3가지
> '구조화된 스트리밍'은 아래의 몇가지 조건을 만족해야만 exactly-once 을 보장합니다

#### 하나. 재수행 가능한 스트리밍 소스 - Replayable streaming sources
* 과거에 실패했던 작업의 위치에서 동일한 작업의 범위의 데이터를 다시 읽어올 수 있을 것
  - 카프카의 경우 데이터가 삭제되지만 않았다면, 오프셋 정보를 통해서 특정 시점 이후의 데이터를 가져올 수 있습니다
  - 예제의 소켓 버서의 경우 한 번 수신된 데이터를 다시 수신할 수 없기 때문에 exactly-once 를 보장하지 못합니다

#### 둘. 결정론적인 계산 - Deterministic computations
* 입력 조건이 동일한 경우, 항상 결과가 동일한 연산이 수행될 수 있는 변환작업일 것
  - 동일한 키가 발생할 수 있는 컬럼을 이용하는 정렬의 경우 여러번 실행하는 경우 결과가 흔들릴 수 있습니다 (분산환경임을 유의할 것)

#### 셋. 멱등한 스트리밍 싱크 - Idempotent streaming sink
* '마이크로 배치'작업이 여러번 수행 되더라도, 항상 동일한 결과를 가질 수 있을 것
  - 반복 수행을 통해 일부의 데이터만 반복 수행되더라도, 항상 Upsert 하거나, Truncate 혹은 Range Delete 후, Insert 하도록 구현하면 멱등성을 보장할 수 있습니다



### 호환성을 보장하는 *감내할 수 있는 변경* 수준 3가지
> '구조화된 스트리밍'의 경우 아래와 같이 체크포인트 상태에 영향을 주지 않는 수준의 마이너한 수정후 실행이 가능합니다 

#### 하나. 특정 라인을 필터 하는 함수 적용
* 문제가 발생할 가능성이 있는 로우를 제거하는 예제
```python
spark> filteredLines = lines.filter("isCorrupted(value) = false")
spark> words = filteredLines.select(split(col("value"), "\s").alias("word"))
```

#### 둘. 소스와 싱크 옵션의 변경
* `host, port 옵션은 변경되어서는 안`되지만, 출력 로우의 수 등의 옵션은 변경할 수 있습니다
```python
spark> writerStream.format("console").option("numRows", "100")
```

#### 셋. 데이터 처리 상세 옵션
* `체크포인트 위치는 변경되어서는 안`되지만, 트리거링 간격과 같은 옵션들은 변경할 수 있습니다


### 4-3. 실행중인 쿼리 모니터링 하기 - Monitoring an Active Query
> '구조화된 스트리밍' 특성상, 실행중인 쿼리를 모니터링하는 기능은 아주 중요하며 다양한 방법을 통해 모니터링 할 수 있습니다

#### StreamingQuery 객체를 이용한 현재 상태 조회하는 두 가지 방법 - Querying current status using StreamingQuery

##### 1. StreamingQuery 객체를 이용한 현재 매트릭 가져오기 - Get current metrics using StreamingQuery
* lastProgress() 메소드를 통해 StreamingQueryProgress(dictionary) 객체를 통해 확인합니다


In [14]:
# 필요한 라이브러리를 임포트합니다
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [35]:
# 소켓 서버로부터 단어를 가져와서 출력하는 가장 간단한 예제를 통해 모니터링 합니다
checkpointDir = "tmp/checkpoint"
!rm -rf $checkpointDir
spark = SparkSession.builder.appName("simple-spark-streaming").config("spark.sql.session.timeZone", "Asia/Seoul").getOrCreate()
reader = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()
counter = reader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("word").count().alias("count")
writer = counter.writeStream.queryName("query-progress").format("console").outputMode("complete")
checkpointDir="tmp/checkpoint"
trigger = writer.trigger(processingTime="1 second").option("checkpointLocation", checkpointDir)
query = trigger.start()
query.awaitTermination(10)
progress = query.lastProgress

In [36]:
from IPython.display import JSON
JSON(progress)

<IPython.core.display.JSON object>

##### lastProgress 컬럼에 대한 설명
| 컬럼 | 설명 |
| --- | --- |
| id | Unique Identifier - 체크포인트 위치와 1:1 매칭되는 유일한 구분자로, 체크포인트 경로가 삭제되기 전까지는 동일한 값이 유지됩니다 |
| runId | Unique Identifier - 현재 (지)시작된 쿼리 인스턴스를 가리키는 구분자이며, 매 실행시마다 변경됩니다 |
| numInputRows | 마지막 '마이크로 배치' 작업에 수행 했던 입력 로우의 수 |
| inputRowsPerSecond | 데이터 소스로부터 입력 로우 수를 말하며, 마지막 수행된 '마이크로 배치' 평균 소요시간을 기준으로 계산 됩니다 |
| processedRowsPerSecond | 데이터 싱크로 처리되어 저장되는 로우의 비율을 말합니다. `입력 로우의 수 대비 처리하는 로우의 수가 일정하게 낮다면` 지연되고 있다고 말할 수 있습니다 | 
| sources and sink | 데이터 소스와 싱크에 대한 정보 |

* status() 함수를 통한 현재 상태 가져오기 - Get current status using StreamingQuery.status().

In [37]:
# 소켓 서버로부터 단어를 가져와서 출력하는 가장 간단한 예제를 통해 모니터링 합니다
status = query.status
query.stop()

In [38]:
JSON(status)

<IPython.core.display.JSON object>

##### 2. 드랍 위자드 메트릭을 이용한 매트릭 게시하기 - Publishing metrics using Dropwizard Metrics
> 스파크는 [Dropwizard Metrics](https://metrics.dropwizard.io/4.2.0/)를 활용하여 리포트 매트릭을 생성하며, 다양한 모니터링 프레임워크(Ganglia, Graphite, JMX 등)을 지원합니다. 기본적으로 출력옵션은 켜져있지 않으므로 필요시에 설정을 해주어야만 합니다. [Dropwizard Metrics for Spark](https://spark.apache.org/docs/latest/monitoring.html#metrics) `spark.sql.streaming.metricsEnabled` 을 통해 설정이 가능합니다.

* 대부분의 모니터링 정보는 작업 모니터링 화면에서 제공하므로, 웹 UI 통한 디버깅을 추천 드립니다.
  - http://localhost:4040/jobs
![spark-streaming-ui](images/spark-streaming-ui.png)

In [39]:
!rm -rf "tmp/checkpoint"
spark = SparkSession.builder.appName("simple-spark-streaming").config("spark.sql.session.timeZone", "Asia/Seoul").getOrCreate()
spark.conf.set("spark.sql.streaming.metricsEnabled", "true")
reader = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()
counter = reader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("word").count().alias("count")
writer = counter.writeStream.queryName("query-dropwizard").format("console").outputMode("complete")
checkpointDir="tmp/checkpoint"
trigger = writer.trigger(processingTime="1 second").option("checkpointLocation", checkpointDir)
query = trigger.start()
query.awaitTermination(30)
query.stop()

#### 커스텀 StreamingQueryListeners 객체를 이용하여 매트릭 게시하기 - Publishing metrics using custom StreamingQueryListeners
> StreamingQueryListener 인터페이스를 통해 다양한 이벤트를 체크할 수 있습니다. 단, 인터페이스 구현을 통한 컴파일 언어만 지원하기 때문에 pyspark 에서는 사용할 수 없습니다.

* 아래의 예제는 3가지 이벤트(시작, 종료, 진행 등)를 모니터링 하는 리스너를 구현합니다
```scala
import org.apache.spark.sql.streaming._
val myListener = new StreamingQueryListener() {
    override def onQueryStarted(event: QueryStartedEvent): Unit = {
        println("Query started: " + event.id)
    }
    override def onQueryTerminated(event: QueryTerminatedEvent): Unit = {
        println("Query terminated: " + event.id)
    }
    override def onQueryProgress(event: QueryProgressEvent): Unit = {
        println("Query made progress: " + event.progress)
    }
}
```

* 구현된 리스너를 SparkSession 실행 전에 등록합니다
```scala
    spark.streams.addListener(myListener)
```

## 5. 스트리밍 데이터 소스와 싱크 - Streaming Data Sources and Sinks

> 기본으로 제공되는 stream 싱크와 소스를 이용하여 데이터 프레임 생성이 가능하며 SparkSession.readStream(), DataFrame.writeStream() 을 이용합니다


### 5-1. 파일 - Files

* 파일로부터 읽기
  - 기본적으로 텍스트 파일(CSV, TSV 등)의 경우 스키마가 없는 경우가 많기 때문에 스키마를 직접 생성해 줍니다


In [9]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import *

source = f"{work_data}/flight-data/json/2010-summary.json"
source = f"{work_data}/json"

In [10]:
df1 = spark.read.json(source)
df1.printSchema() # 기본적으로 스키마를 추론하는 경우 integer 가 아니라 long 으로 잡는다
display(df1)
df1.select("value").groupBy("value").sum()

root
 |-- key: long (nullable = true)
 |-- value: long (nullable = true)



key,value
1,1000
2,2000
1,1000
9,9000


value,sum(value)
9000,9000
1000,2000
2000,2000


In [11]:
checkpointDir = "tmp/simple-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("key", IntegerType()))
    .add(StructField("value", IntegerType()))
)
df = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)
summary = df.select("value").groupBy("value").sum()
writer = (
    summary
    .writeStream
    .format("console")
    .outputMode("complete")
    .option("checkpointLocation", checkpointDir)
    .start()
)
writer.awaitTermination(10)
writer.stop()

#### 파일을 사용할 때에 유의해야 할 특징 4가지
* 하나. 모든 파일들의 포맷과 스키마는 동일해야 하며, 그렇지 않은 경우 쿼리의 실패로 이어지게 됩니다
* 둘. 디렉토리 내의 파일 단위로 원자적으로 동작해야 하며, 한번에 정상적으로 읽어져야 합니다
  - 즉, append 되고 있는 파일의 경우 문제가 될 수 있으며, update 혹은 append 되어서는 안 됩니다
* 셋. 디렉토리 내의 새로운 파일들이 존재하더라도 모든 파일이 동시에 수행되지 않습니다
  - 동시 수행 임계치에 따라 병렬로 수행되며, 정해진 순서를 보장하지 않습니다.
  - maxFilesPerTrigger 옵션을 통해 처리율을 조정할 수 있습니다 - [프로그래밍 가이드](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#creating-streaming-dataframes-and-streaming-datasets)를 참고할 수 있습니다
* 넷. 파일 쓰기는 append 모드만 지원하며, 이미 존재하는 파일에는 append 될 수 없습니다
  - *end-to-end exatly-once gurantees* 지원은 _spark_metadata 경로에 존재하는 log 파일들을 통해 관리되어 가능하며, 종복 혹은 일부만 읽혀지는 경우는 없습니다. 
  - 만일 애플리케이션 재실행 시에 변경된 스키마로 데이터가 추가 되었다면, 쿼리 시에 해당 스키마가 조정되어야만 합니다

### 5-2. 아파치 카프카 - Apache Kafka

![table.8-1](images/table.8-1.png)

#### 1. 카프카에 데이터 쓰기 - producer.py
```python
#!/usr/bin/env python
import sys
from time import sleep
from json import dumps
from kafka import KafkaProducer
# 카프카에 데이터를 전송하는 코드
def produce(port):
    hostname="localhost:%d" % port
    producer = KafkaProducer(
        bootstrap_servers=[hostname],
        value_serializer=lambda x: dumps(x).encode('utf-8')
    )
    for j in range(9999):
        print("Iteration", j)
        data = {'counter': j}
        producer.send('events', value=data)
        sleep(0.5)
# 카프카의 데이터 수신을 위한 포트는 9092
if __name__ == "__main__":
    port = 9092
    if len(sys.argv) == 2:
        port = int(sys.argv[1])
    produce(port)
```
#### 2. 도커 컨테이너를 통해 접속하여 카프카에 메시지를 생성합니다
```bash
$ docker compose exec notebook bash
# python producer.py
```

#### 3. 스파크를 통해 생성된 스트리밍 데이터를 확인합니다

In [3]:
# 컨테이너 내부에서 토픽에 접근하기 위한 포트는 9093
reader = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9093")
    .option("subscribe", "events")
    .load()
)

In [4]:
counter = reader.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [5]:
checkpointDir="tmp/kafka-stream"
!rm -rf "tmp/kafka-stream"
writer = (
    counter
    .writeStream
    .format("console")
    .option("checkpointLocation", checkpointDir)
    .start()
)
writer.awaitTermination(60)
writer.stop()


### 5-3. 커스텀 스트리밍 소스와 싱크 - Custom Streaming Sources and Sinks

> 빌트인으로 지원하지 않는 저장소 엔진(예: Cassandra 등)의 경우 foreachBatch() 혹은 foreach() 메소드를 통해 커스텀 로직을 추가할 수 있습니다.


#### 모든 스토리지 시스템에 쓰기 - Writing to any storage system

> 현재 '스파크 스트리밍'에는 카산드라에 스트리밍 데이터프레임을 저장할 수 있는 방법은 없기 때문에, Batch DataFrame 방식으로 저장합니다

#### 가. foreachBatch : 임의의 작업들 혹은 매 '마이크로 배치'의 결과에 대해 커스텀 로직을 적용할 수 있는 배치 메소드
  - foreachBatch(arg1: DataFrame or Dataset, arg2: Unique Identifier)

* 로컬 파일을 카산드라 데이터베이스에 저장합니다
  - 로컬에 저장된 JSON 파일을 읽어서 스트리밍 처리를 합니다
  - 초기 test_keyspace 와 test_table 에 저정합니다

In [3]:
source = "/home/jovyan/work/data/json"
checkpointDir = "tmp/cassandra-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
)
df = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)

In [4]:
hostAddr = "cassandra"
keyspaceName = "test_keyspace"
tableName = "test_table"
spark.conf.set("spark.cassandra.connection.host", hostAddr)

def writeCountsToCassandra(updatedCountsDF, batchId):
    # Use Cassandra batch data source to write the updated counts
    (
        updatedCountsDF
        .write
        .format("org.apache.spark.sql.cassandra")
        .mode("append")
        .options(table=tableName, keyspace=keyspaceName)
        .save()
    )

In [5]:
summary = df.select("emp_id", "emp_name")
streamingQuery = (
    summary
    .writeStream
    .foreachBatch(writeCountsToCassandra)
    .outputMode("update")
    .option("checkpointLocation", checkpointDir)
    .start()
)

* 카산드라에 접속하여, 데이터를 조회합니다
```bash
$> docker compose exec cassandra cqlsh
cqlsh> use test_keyspace;
cqlsh:test_keyspace> select emp_id, emp_name from test_table;
```

#### foreachBatch() 로 아래의 동작들이 가능합니다

* 이전에 존재하는 배치 데이터 소스들을 재사용 하기 - *Reuse existing batch data sources* 
  - 이전 예제와 같이 배치에서 사용하는 데이터 소스들을 그대로 사용할 수 있습니다

* 다수의 경로에 저장하기 - *Write to multiple locations*
  - OLAP 및 OLTP 데이터베이스 등에 다양한 위치로 저장할 수 있습니다
  - 출력 데이터에 대해 데이터프레임 캐시를 통해 재계산을 회피할 수 있습니다

```python
# In Python
def writeCountsToMultipleLocations(updatedCountsDF, batchId):
    updatedCountsDF.persist()
    updatedCountsDF.write.format(...).save() # Location 1
    updatedCountsDF.write.format(...).save() # Location 2
    updatedCountsDF.unpersist()
}
```

* 추가적인 데이터프레임 연산 하기 - *Apply additional DataFrame operations*
  - Incremental Plan 생성이 어렵기 때문에 '스트리밍 데이터프레임'에 대한 연산 지원이 되지 않습니다 - **아래**
  - '마이크로 배치' 출력에 대하여 foreachBatch 메소드를 통해 작업 수행이 가능합니다.
  - **at-least-once** 지원만 가능하며, **exactly-once** 는 후처리 dedup 작업을 통해 가능합니다


#### '스파크 스트리밍'에 지원되지 않는 연산 - Unsupported Operations

> 스트리밍 '데이터프레임/데이터셋' 에서는 몇 가지 지원하지 않는 연산이 있습니다. 실행 시에는 ***DataFrame/Datasets 에서 지원하지 않는 연산이라는 AnalysisException*** 을 반환합니다. 생각해보면 끝없이 수신되는 데이터에 대한 정렬을 하는 것은 모호한 동작이며, 수신된 데이터를 모두 모니터링 해야 하므로, 비효율적인 연산일 가능성이 높습니다.

| 연산자 | 설명 |
| --- | --- |
| 다수 스트리밍 집계 연산 | 여러개의 스트리밍 데이터프레임의 집계는 현재 지원하지 않습니다 |
| limit or take | 스트리밍 데이터셋은 일부 로우를 가져오는 연산은 지원하지 않습니다 |
| distinct | 스트리밍 데이터셋은 유일 데이터 연산은 지원하지 않습니다 |
| sort | 집계 연산 이후의 Complete 출력모드에서만 정렬을 사용할 수 있습니다 |
| outer join | 스트림 사이드에 대해서만 left inner, outer 조인을 지원 |
| count | 단일 빈도수를 반환하지 않기 때문에, 실행 횟수를 반환하는 ds.groupBy().count() 를 사용하세요 |
| foreach | 대신 ds.writeStream.foreach(...) 를 사용하세요 (다음 챕터에서) |
| show | 콘솔 sink 를 사용하세요 (다음 챕터에서) |
| cube, rollup | 연산 비용 및 실용성 면에서 지원하지 않는 연산 UnsupportedOperationException |

* [support-matrix-for-joins-in-streaming-queries](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#support-matrix-for-joins-in-streaming-queries)
  - 양쪽이 모두 스트림인 경우는 워터마크를 사용해야만 합니다


#### 나. foreach : 커스텀 저장 로직을 매 로우마다 적용하는 메소드 (고정된 map 함수 같은 방식)
> 배치저장을 위한 인터페이스가 제공되지 않는 경우 직접 데이터를 저장하는 로직을 구현해야 하며, open, process, close 단계에 해당하는 구현을 해야만 합니다


In [14]:
source = "/home/jovyan/work/data/json"
checkpointDir = "tmp/foreach-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
)
df = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)

In [38]:
# 화면에 데이터를 그대로 출력하는 예제
def process_row(row):
    print("%d, %s" % (row[0], row[1]))
query = df.writeStream.foreach(process_row).start()

In [35]:
import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(user='sqoop', password='sqoop', host='mysql', database='testdb')
cursor = cnx.cursor()

def createEmployee(cursor):
    try:
        employee = "CREATE TABLE employees ( emp_id int, emp_name varchar(30) )"
        cursor.execute(employee)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            return "employees table already exists."
        else:
            return err.msg
    else:
        return "employees table created."

print(createEmployee(cursor))

def deleteEmployee(delete_employee):
    try:
        cursor.execute(delete_employee)
    except:
        import traceback
        traceback.print_exc()
        return "failed to delete employees"
    return "employees table deleted."

delete_employee = "DELETE FROM employees"
print(deleteEmployee(delete_employee))

def insertEmployee(add_employee, data_employee):
    emp_no = -1
    try:
        cursor.execute(add_employee, data_employee)
        emp_no = cursor.lastrowid
    except:
        import traceback
        traceback.print_exc()
        return emp_no
    return emp_no

add_employee = "INSERT INTO employees ( emp_id, emp_name ) VALUES ( %s, %s )"
data_employee = ( 1, '박수혁' )
print(insertEmployee(add_employee, data_employee))

cnx.commit()
cursor.close()
cnx.close()

employees table already exists.
employees table deleted.
0


#### 저장된 데이터를 확인합니다
```bash
$ docker compose exec mysql mysql -usqoop -psqoop
mysql> use testdb;
mysql> select emp_id, emp_name from employees;
+--------+--------------------------+
| emp_id | emp_name                 |
+--------+--------------------------+
|      1 | 엘지전자                   |
|      2 | 엘지화학                   |
|      3 | 엘지에너지솔루션             |
|    100 | 엘지                      |
+--------+--------------------------+
```

#### ForeachWriter class 를 생성하고 외부 데이터베이스에 저장하는 예제
> 아래의 경우는 foreach 메소드를 사용하는 예제를 설명하기 위함이며, 실무에서는 ***절대 사용해서는 안 되는 예제*** 입니다.

In [60]:
import mysql.connector
from mysql.connector import errorcode
class ForeachWriter:    
    
    def createEmployee(self, cursor):
        try:
            employee = "CREATE TABLE employees ( emp_id int, emp_name varchar(30) )"
            cursor.execute(employee)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                return "employees table already exists."
            else:
                return err.msg
        else:
            return "employees table created."
        
    def deleteEmployee(self, cursor):
        try:
            delete_employee = "DELETE FROM employees"
            cursor.execute(delete_employee)
        except:
            import traceback
            traceback.print_exc()
            return "failed to delete employees"
        return "employees table deleted."
        
    def insertEmployee(self, cursor, data_employee):
        emp_no = -1
        try:
            add_employee = "INSERT INTO employees ( emp_id, emp_name ) VALUES ( %s, %s )"
            cursor.execute(add_employee, data_employee)
            emp_no = cursor.lastrowid
        except:
            import traceback
            traceback.print_exc()
            return emp_no
        return emp_no
    
    def insertDocument(self, data_employee):
        emp_id = -1
        try:
            cnx = mysql.connector.connect(user='sqoop', password='sqoop', host='mysql', database='testdb')
            cursor = cnx.cursor()
            emp_id = self.insertEmployee(cursor, data_employee)
            cnx.commit()
        except:
            import traceback
            traceback.print_exc()
        finally:
            cursor.close()
            cnx.close()
        return emp_id
    
    def selectEmployee(self, cursor):
        emp_no = -1
        try:
            get_employees = "SELECT emp_id, emp_name FROM employees"
            cursor.execute(get_employees)
            for (emp_id, emp_name) in cursor:
                print("{} in '{}' has retrieved.".format(emp_id, emp_name))
        except:
            import traceback
            traceback.print_exc()
            return emp_no
        return emp_no
    
    # 데이터베이스 커넥션, 테이블 생성 및 테이블 데이터 삭제
    def open(self, partitionId, epochId):
        try:
            cnx = mysql.connector.connect(user='sqoop', password='sqoop', host='mysql', database='testdb')
            cursor = cnx.cursor()
            print(self.createEmployee(cursor))
            print(self.deleteEmployee(cursor))
            cnx.commit()
        except:
            import traceback
            traceback.print_exc()
        finally:
            cursor.close()
            cnx.close()
        return True
    
    def process(self, row):
        data_employee = (row[0], row[1])
        result = self.insertDocument(data_employee)
        if (result >= 0):
            print("[%s] '%s' is inserted" % data_employee)
    
    def close(self, error):
        try:
            cnx = mysql.connector.connect(user='sqoop', password='sqoop', host='mysql', database='testdb')
            cursor = cnx.cursor()
            self.selectEmployee(cursor)
        except:
            import traceback
            traceback.print_exc()
        finally:
            cursor.close()
            cnx.close()

df.writeStream.foreach(ForeachWriter()).start()

## 6. 데이터 변환 - Data Transformations

> '구조화된 스트리밍' 에서는 점진적으로 수행 가능한 데이터프레임 연산만 지원하며, 이를 크게 '스테이트리스', '스테이트풀' 연산이라 합니다

### 6-1. 증분 실행과 스트리밍 처리상태 - Incremental Execution and Streaming State

> 한 번에 모든 '논리적 계획'을 '물리적계획'으로 변환하는 배치 처리와 다르게, 연속적인 실행의 계획을 생성하는 역할을 수행합니다. 매 실행은 '마이크로배치' 작업을 말하며, **작업과 작업 간의 중간 결과물**을 스트리밍 **'상태'**라고 말합니다.

* '스테이트리스' 데이터 변환 - Stateless Transformations
  - 모든 프로젝션 연산(select, explode, map, flatMap) 들과 셀렉트 연산(filter, where) 들은 이전 로우의 상태와 무관한 '스테이트리스' 연산자들입니다
  - 이러한 **'스테이트리스' 연산자 들만 append, update 출력 모드를 지원**합니다.
    - '중간결과물인 상태'가 없다는 의미는 현재 로우에만 영향 및 의존성을 가진다는 말이므로, append 모드로 동작 하더라도 중복문제가 생기지 않습니다
  - **반면에 complete 모드를 지원하지 않습**니다,
    - 이는 매 새로운 데이터를 반복적으로 처리하는 것은 비용적으로 크기 때문입니다.

* '스테이트풀' 데이터 변환 - Stateful Transformations
  - DataFrame.groupBy().count() 스트리밍의 시작부터 발생하는 레코드의 수를 나타내는데, 이전 '마이크로배치'의 빈도수를 저장하고 있어야 하며 이를 '상태'라고 부릅니다
  - 상태저장이 어떻게 동작하는 지 테스트 하는데, 파일 스트림을 통한 연속적인 테스트는 어렵다는 사실을 알 수 있습니다

In [14]:
checkpointDir = "tmp/stateful-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
)
source = "/home/jovyan/work/data/tmp"
reader = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)
groupby = reader.groupBy().count()
writer = (
    groupby
    .writeStream
    .format("console")
    .outputMode("complete")
    .option("checkpointLocation", checkpointDir)
    .start()
)
writer.awaitTermination(120)
writer.stop()

* 해서 이번에는 소켓을 통해 지속적으로 넣어주면서 groupBy count 를 살펴봅니다
```bash
$ nc -lvp 9999
```

In [12]:
reader = (
    spark
    .readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .load()
)
groupby = reader.groupBy().count()
writer = (
    groupby
    .writeStream
    .format("console")
    .outputMode("complete")
    .start()
)
writer.awaitTermination(120)
writer.stop()

#### 분산 내결함성 상태 관리 - Distributed and fault-tolerant state management
* 드라이버에서 기동되는 스파크 스케줄러는 애플리케이션을 잡으로 잡을 다수의 타스크로 나누고, 이러한 타스크들을 큐에 집어 넣습니다
* 클러스터에 여유 리소스가 존재하는 경우 익스큐터는 기동되어 타스크가 저장된 큐로부터 작업을 가져와서 수행합니다
* 스트리밍 쿼리의 매 '마이크로배치'는 주어진 집합의 데이터를 '스트리밍 소스'들로 부터 가져와서, 처리한 후, '스트리밍 싱크'에 저장합니다
* 이 과정에서 생성되는 '중간 상태 데이터'들은 다음 '마이크로배치' 작업에서 소비되게 됩니다

![figure.8-6](images/figure.8-6.png)

* 집계를 위해 익스큐터 내에서 셔플링이 일어나고, 익스큐터 내의 메모리에서 캐싱이 일어납니다
* 해당 단어는 항상 동일한 익스큐터 내에서 수행되므로, 로컬 읽기와 갱신이 이루어지게 됩니다 (파티셔닝 개념)
* 상태저장을 위한 메모리가 부족하거나, 익스큐터가 예기치 못한 상황에 종료되는 경우에 체크포인트 위치에 상태의 체인지로그를 키값 쌍으로 저장합니다
  - 만에 하나 실패하는 경우에도 해당 체크포인트의 체인지로그를 통해서 동일한 상태로 다시 수행하므로써 복구 가능합니다.
  - 이러한 과정을 통해서 **end-to-end exactly-once 를 보장**합니다


#### 스테이트풀 연산자의 유형 - Types of stateful operations

* 매니지드 스테이트풀 연산자 - Managed stateful operations
  - 구체적으로 "오래된" 것으로 정의된 연산을 기반으로, 자동으로 오래된 상태를 구별하고 정리합니다
    - Streaming aggregations
    - Stream-stream joins
    - Streaming deduplication

* 언매니지드 스테이트풀 연산자 - Unmanaged stateful operations
  - 이용자 스스로 정리 로직을 구현하는 '세션처리'와 같은 임의의 상태저장을 말합니다
    - MapGroupsWithState
    - FlatMapGroupsWithState


## 7 상태 저장 스트리밍 집계 - Stateful Streaming Aggregations

### 7-1. 시간에 기반하지 않는 집계 - Aggregations Not Based on Time

#### Global aggregations - 스트림 데이터 전역적인 집계함수
* 스트리밍의 경우 **데이터프레임에 직접 빈도 함수를 호출할 수 없습**니다 (bounded vs. unbounded)
```python
runningCount = sensorReadings.groupBy().count()
```

#### Grouped aggregations - 특징 그룹필드에 대한 집계함수
```python
baselineValues = sensorReadings.groupBy("sensorId").mean("value")
```

#### [All built-in aggregation functions](https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#aggregate-functions)
  - sum, mean, stddev, countDistinct, collect_set, approx_count_distinct 등

#### Multiple aggregations computed together - 다수의 집계 연산
```python
multipleAggs = (
    sensorReadings
    .groupBy("sensorId")
    .agg(
        count("*")
        , mean("value").alias("baselineValue")
        , collect_set("errorCode").alias("allErrorCodes")
    )
)
```

#### [User-defined aggregation functions](https://spark.apache.org/docs/latest/sql-ref-functions-udf-aggregate.html#untyped-user-defined-aggregate-functions)
* 이용자 정의 집계함수는 Jar 형태로 배포 및 등록 되어야 하므로, Scala, Java 언어만 지원하며, 등록된 UDF, UDAF 등을 SQL에서 사용할 수 있습니다
![ch8-udf](images/ch8-udf.png)

#### 시간을 기준으로 하지 않는 집계의 경우 유의해야 할 2가지
* ***1. Output Mode*** : 워터마크를 사용할 수 없으므로 출력 모드가 제한적이다
* ***2. Planning the resource usage by state*** : 워터마크 사용이 불가하므로 특정 상태에 과도한 메모리 사용이 발생할 수 있다



### 7-2. 이벤트타임 윈도우 기반 집계 - Aggregations Not Based on Time

> 수신되는 전체 데이터의 집계 대신 '타임윈도우'에 근거한 집계를 하려고 합니다. 예를 들어 비정상적으로 많은 빈도를 나타내는 센서를 인식하는 등의 임의의 단위시간 내의 빈도를 측정할 수 있습니다

* 애플리케이션의 강건성을 확보하기 위하여, 센서 데이터가 단위 시간 당 생성되는 빈도 혹은 양을 기준으로 인터벌을 정해야만 합니다. 
* 실제 사건이 발생한 시간을 나타내는 *event time* 기준으로 데이터의 타임 윈도우를 측정합니다
  - 아래의 예제에서는 eventTime 컬럼을 기준으로 5분 단위 윈도우를 구성하여 계산합니다
```python
from pyspark.sql.functions import *
(
    sensorReadings
    .groupBy("sensorId", window("eventTime", "5 minute"))
    .count()
)
```
* 데이터가 발생한 시간기준으로 단위시간 5분 내 이벤트를
* sensorId 키를 기준으로 복합 그룹을 통해 계산하며
* 복합 그룹 내의 빈도를 갱신합니다


#### 5분 텀플링 윈도우 (nonoverlapping)

![figure.8-7](images/figure.8-7.png)



In [5]:
checkpointDir = "checkpoint/tumbling-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
    .add(StructField("timestamp", TimestampType()))
    .add(StructField("time", StringType()))
)
source = "data/stream/tumbling"
reader = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)
groupby = reader.groupBy("emp_id", window("timestamp", "5 minute")).count()
writer = (
    groupby
    .writeStream
    .format("console")
    .outputMode("complete")
    .option("checkpointLocation", checkpointDir)
    .option("truncate", "false")
    .start()
)
writer.awaitTermination(10)
writer.stop()

> 텀블링 윈도우 데이터 생성 및 결과 확인을 위해 예제 데이터를 생성하고 파일스트리밍 결과를 확인합니다. 집계함수의 complete 모드이지만 정렬은 되지 않았으나, 정해진 타임슬롯에 전체 데이터를 출력할 수 있었습니다

```bash
notebook  | -------------------------------------------
notebook  | Batch: 0
notebook  | -------------------------------------------
notebook  | +------+------------------------------------------+-----+
notebook  | |emp_id|window                                    |count|
notebook  | +------+------------------------------------------+-----+
notebook  | |1     |[2016-03-20 15:05:00, 2016-03-20 15:10:00]|1    |
notebook  | |4     |[2016-03-20 12:50:00, 2016-03-20 12:55:00]|2    |
notebook  | |2     |[2016-03-20 12:35:00, 2016-03-20 12:40:00]|1    |
notebook  | |3     |[2016-03-20 11:45:00, 2016-03-20 11:50:00]|1    |
notebook  | |1     |[2016-03-20 12:20:00, 2016-03-20 12:25:00]|3    |
notebook  | +------+------------------------------------------+-----+
```


![figure.8-8](images/figure.8-8.png)

> 이전 윈도우와 오버랩되는 구간이 발생하는 슬라이딩 윈도우 (윈도우는 5분이고, 윈도우 간에 5분씩 오버랩)


In [7]:
checkpointDir = "checkpoint/sliding-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
    .add(StructField("timestamp", TimestampType()))
    .add(StructField("time", StringType()))
)
source = "data/stream/sliding"
reader = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)
groupby = reader.groupBy("emp_id", window("timestamp", "10 minute", "5 minute")).count()
writer = (
    groupby
    .writeStream
    .format("console")
    .outputMode("complete")
    .option("checkpointLocation", checkpointDir)
    .option("truncate", "false")
    .start()
)
writer.awaitTermination(10)
writer.stop()

> 슬라이딩 윈도우의 경우 5분 오버랩 윈도우를 통해 겹치는 구간이 존재하도록 출력됩니다 

```bash
notebook  | -------------------------------------------
notebook  | Batch: 0
notebook  | -------------------------------------------
notebook  | +------+------------------------------------------+-----+
notebook  | |emp_id|window                                    |count|
notebook  | +------+------------------------------------------+-----+
notebook  | |3     |[2016-03-20 11:40:00, 2016-03-20 11:50:00]|1    |
notebook  | |1     |[2016-03-20 12:20:00, 2016-03-20 12:30:00]|3    |
notebook  | |1     |[2016-03-20 12:15:00, 2016-03-20 12:25:00]|3    |
notebook  | |4     |[2016-03-20 12:45:00, 2016-03-20 12:55:00]|2    |
notebook  | |4     |[2016-03-20 12:50:00, 2016-03-20 13:00:00]|2    |
notebook  | |3     |[2016-03-20 11:45:00, 2016-03-20 11:55:00]|1    |
notebook  | |1     |[2016-03-20 15:05:00, 2016-03-20 15:15:00]|1    |
notebook  | |2     |[2016-03-20 12:30:00, 2016-03-20 12:40:00]|1    |
notebook  | |1     |[2016-03-20 15:00:00, 2016-03-20 15:10:00]|1    |
notebook  | |2     |[2016-03-20 12:35:00, 2016-03-20 12:45:00]|1    |
notebook  | +------+------------------------------------------+-----+
```

> 이와 같이 누적으로 모든 데이터를 담고 있다면 리소스 사용 관점에서 지속적으로 늘어나는 상태를 관리하기 어려울 것이므로, 최신 데이터만 유지하되, 어느정도 까지 데이터 입력지연을 감내할 것인지를 정의해야 합니다. 이러한 쿼리의 바운드의 범위를 ***watermarks*** 라고 부릅니다.

#### Handling late data with watermarks
> ***watermark***는 처리된 데이터 내의 쿼리에 의해 발견된 **maximum event time** 이후에 발생하는 **이벤트 타임의 임계치 값**이라고 말할 수 있습니다.

* 주어진 그룹에 더 이상 데이터가 도달하지 않는다는 것을 아는 시점을 말하며, 엔진에 의해서 자동적으로 해당 집계가 완료됨을 인지할 수 있습니다.


![figure.8-9](images/figure.8-9.png)


In [17]:
checkpointDir = "checkpoint/watermark-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
    .add(StructField("timestamp", TimestampType()))
    .add(StructField("time", StringType()))
)
source = "data/stream/watermark"
reader = (
    spark
    .readStream
    .format("json")
    .schema(schema)
    .load(source)
)
groupby = (
    reader
    .withWatermark("timestamp", "5 minutes")
    .groupBy("emp_id", window("timestamp", "30 minute", "30 minute"))
    .count()
)
writer = (
    groupby
    .writeStream
    .format("console")
    .outputMode("complete")
    .option("checkpointLocation", checkpointDir)
    .option("truncate", "false")
    .start()
)
writer.awaitTermination(10)
writer.stop()

> 파일스트림의 경우 하나의 파일내의 순서와 무관하게 병렬처리하게 되어 한 번에 동시에 온 것처럼 인식되어 워터마크와 딜레이 영향을 받지 않는 것 처럼 보인다

* 예를 들어 센서 데이터는 아무리 늦어도 10분 이상 지연되지 않는다는 것을 안다면, **워터마크를 10분으로** 지정합니다
  - 반드시 groupBy 집계 이전에 withWatermark 선언이 되어야만 합니다.
  - 쿼리가 수행될 때에 '구조화된 스트리밍'은 ***이벤트 타임***의 최대 값을 계속해서 관찰합니다
  - 워터마크를 갱신함에 따라서 ***너무 늦은*** 데이터를 필터할 수 있게 됩니다
  - 그리고나서 '오래된 상태'를 정리하게 됩니다. 여기서 ***10분 이상 지연된 어떠한 데이터도 무시***된다는 의미입니다
  - 이벤트 타임 기준으로 가장 늦게 발생한 입력 데이터 보다 ***10분 이상 오래된 모든 타임 윈도우는 상태에서 정리***되게 됩니다

```python
df = (
    sensorReadings
    .withWatermark("eventTime", "10 minutes") # 최대 10분까지 지연 로그를 대기한다는 의미
    .groupBy("sensorId", window("eventTime", "10 minutes", "5 minutes")) # window(windowLength: '타임 윈도우 크기', slideInterval: '오버랩 시간')
    .mean("value")
)
```

#### 윈도우 그룹 빈도수에 대한 워터마킹 동작 방식

* 매 5분 단위로 트리거링이 발생하며, 집계연산이 수행됩니다
* X축은 Processing Time 즉, Scheduler 를 통해 트리거링이 발생하는 '프로세싱 타임'을 말하며, Y축은 실제 사건이 발생한 '이벤트 타임'을 가리킵니다
* 맨 처음 발생하는 12:05 분에는 어떠한 데이터도 수신되지 않았기 때문에 '쿼리 테이블'에는 아무런 데이터도 없습니다
* 12:10 트리거링 시에는 2건의 데이터가 수신되었고, '이벤트 타임'이 각 각 "12:07,id1 12:08,id2" 으로 사건이 발생한 시간과 수신된 시간이 거의 동일합니다
  - X, Y 축의 위치가 일치한 점으로 알 수 있으며 기울기가 1인 위치에 있는 데이터는 지연이 거의 없다고 말할 수 있습니다
* 12:10 트리거링 시에 윈도우가 10분이고, 슬라이딩이 5분이기 때문에 2개의 '타임 슬라이드' 그룹에 대한 계산이 이루어져야 합니다
  - "12:00 ~ 12:10" 과 "12:05 ~ 12:15" 2개가 그것입니다
* 12:15 트리거링 시에는 지연된 데이터 "12:09,id3"가 도착했고, 워터마크 내에 존재하여 '쿼리 테이블'에 반영되어집니다
  - 워터마크 계산은 '프로세싱 타임' 기준으로 가장 최신에 수신된 '이벤트 타임' - '12:14,id2' 기준으로 10분 과거 시간까지 허용합니다
  - 즉, watermark = 12:14 - 10mins = 12:04 분이전 데이터는 지연된 데이터로 버리고, 그 이후 데이터는 워터마크 내의 데이터입니다
* 12:10 트리거링 시에는 총 4개의 데이터가 수신되었고, 2개는 정상, 2개는 지연된 데이터가 존재합니다만, 버리는 경우는 없습니다
  - 지연을 결정짓는 가장 큰 기준은 절대적인 시간이 아니며, ***수신된 데이터의 maximum(eventTime) 시간을 기준으로 계산***합니다
* 현재 '프로세싱 타임'이 아무리 흘러도, 수신되는 '이벤트 타임' 시간이 천천히 흘러간다면 지연이 아니게 됩니다 
  - Y 축으로 늘어나는 시간은 '이벤트 타임' 기준으로 거리가 일정하고 정확하지만, 상대적으로 X 축은 그렇지 못 합니다
  - 일례로 "12:07 ~ 12:08" 과 "12:14 ~ 12:15" 의 Y 축의 거리는 일정하고 동일하지만, X 축의 거리는 전자가 다소 짧습니다 (상대적인 워터마크 계산임을 알아야 합니다)
* '스파크 스트리밍'은 그림에서의 ***파랑색 점선*** 을 항상 모니터링 하면서 '워터마크'를 계산합니다
  - 항상 가장 최신의 '이벤트 타임'을 저장하고 있다가, 트리거링 되는 시점에 watermark 시간을 계산하여 너무 지연된 데이터를 필터링 하게 됩니다
  - 결국, **'쿼리 테이블'도 '이벤트 타임'기준으로 슬라이딩 하면서 갱신**한다고 보면 됩니다
  
  
> ***워터마킹이 적용된 스트리밍 처리는 가장 최신 이벤트 타임 기준으로 워터마크 시간(그림은 10분)만큼 지연된 데이터까지 처리하는 것을 보장합니다*** 
최대 지연 시간의 데이터는 확실히 보장하지만, 누락되지 않는 방향은 확실히 보장하지만, 반면에 ***지연이 되었다고 해서 반드시 누락되는 것을 보장하지는 않습니다.***
이러한 누락되도 되는 데이터를 집계하는 것은 *레코드가 수신된 시점과, 마이크로 배치 처리가 트리거링된 시점*에 기인합니다.


![figure.8-10](images/figure.8-10.png)


In [14]:
from pyspark.sql.functions import explode

checkpointDir = "checkpoint/kafka-watermark-stream"
!rm -rf $checkpointDir
schema = (
    StructType()
    .add(StructField("emp_id", IntegerType()))
    .add(StructField("emp_name", StringType()))
    .add(StructField("timestamp", TimestampType()))
    .add(StructField("time", StringType()))
)
# 컨테이너 내부에서 토픽에 접근하기 위한 포트는 9093
reader = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9093")
    .option("subscribe", "events")
    .option("startingOffsets", "latest") # earliest
    .load()
)
# counter = reader.selectExpr("CAST(value AS STRING)")
# counter = reader.select(from_json(col("value").cast("string"), schema).alias("json"))

gropuby = (
    reader
    .select(from_json(col("value").cast("string"), schema).alias("x"))
    .selectExpr("x.emp_id as emp_id", "x.timestamp as timestamp")
    .withWatermark("timestamp", "25 minutes")
    .groupBy("emp_id", window("timestamp", "25 minute", "25 minute"))
    .count()
)
query = (
    gropuby
    .writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .start()
)
query.awaitTermination(30)
query.stop()

#### 지원하는 출력 모드 - Supported output modes

> '시간을 포함하지 않은 스트리밍 집계'와는 다르게 '이벤트 타임을 포함한 스트리밍 집계'는 3가지 모드를 모두 지원합니다

* Update mode
  - 매 '마이크로 배치'집계 결과에서 변경된 사항만 출력으로 생성되고, 워터마킹은 주기적으로 '상태'를 갱신합니다
  - 하지만, ***Parquet, ORC 와 같은 파일 기반의 싱크의 경우는 '업데이트 모드'를 사용할 수 없***습니다 (단, Delta Lake 는 제외)
  - "수시로 변경 사항만을 지속적으로 갱신하는 경우"

* Complete mode
  - 시간 혹은 변화에 무관하게 모든 출력을 항상 만들어내고 갱신합니다
  - 이 모드는 ***워터마크를 명시하였다고 하더라도 '상태'는 클린업 되지 않으며, 모든 과거의 '상태'를 유지***합니다.
  - 이러한 이유로 무한히 상태가 증가하여 메모리 사용에 항상 주의해야만 합니다

* Append mode
  - ***event-time window 와 watermarking 을 사용했을 때***에만 이용 가능한 모드이며, 이전 결과(상태)에 대해 수정이 불가합니다
  - *워터마크를 명시하지 않은 집계*의 경우는 미래의 데이터로 사용되므로, append 모드에서 *출력이 불가*능합니다
  - 즉, *워터마크가 명시되어야만, 해당 집계 그룹 내의 데이터가 더 이상 업데이트 되지 않음을 알 수* 있습니다
  - append 모드의 경우 '키'와 '집계결과'가 더 이상 과거의 데이터를 갱신하지 않는다는 것을 보장하는 경우에만 *최종 집계 결과를 출력*합니다
  - 이러한 이유로, ***파일과 같은 싱크에 append-only** 출력이 가능하지만, 반면에 워터마크 시간 만큼 ***실시간 처리가 지연***되게 됩니다


## 8 스트리밍 조인 - Streaming Joins

> 스트리밍 데이터 집합간의 조인에 대해 학습합니다. 어떤 조인(이너, 아우터)이 지원되며, 스테이트풀 조인을 위해 저장되는 '상태'를 워터마크를 이용하여 제한하는 방법에 대해 배웁니다.

### 8-1. 스트림 정적 테이블 조인 - Stream–Static Joins

> 데이터 로그에 모든 정보를 다 담을 수 없기 때문에, 로그의 타입과 그 정보를 가진 맵핑 테이블을 통해 집계 정보를 통해 집계 정보를 보여줄 수 있습니다. 광고 클릭 로그(stream)와 광고 당 발생하는 노출정보(static)를 이용하여 실시간 수익을 계산할 수 있습니다. 즉, 자주 변경되지 않으며 사전에 계획된 광고 노출 데이터(static)와의 조인이 필요한 경우에 사용하면 상대적으로 성능과 스트리밍의 효과를 누릴 수 있습니다

```python
impressionStatic = spark.read.parquet("/static/impression/dt=20210707")
clickStream = spark.readStream.format("kafka").load()
# 광고 구분자를 조인 키로 조인을 수행합니다 - stream inner-join static
matched = clickStream.join(impressionStatic, "adId")
# 혹은 stream left-outer-join 혹은 right-outer-join stream 만 지원합니다
matchedOuter = clickStream.join(impressionStatic, "adId", "leftOuter")
```

#### 스트림 조인에서 유의해야 할 사항
* 스트림-정적 조인의 경우 스테이트리스 연산이므로, '워터마킹'을 필요로 하지 않습니다
  - 워터마크는 스트림 처리의 중간 결과 상태 데이터를 말합니다
* 정적 데이터의 경우 반복적으로 읽게 되므로 캐싱하여 성능을 높일 수 있습니다
* 정적 데이터가 변경되는 경우 소스의 종류에 따라 반영되지 않을 수 있습니다
  - 파일 소스에 데이터가 추가(append)되는 경우 스트리밍 쿼리가 재시작 되기 전까지는 반영되기 어렵습니다


### 8-2. 스트림 스트림 테이블 조인 - Stream–Stream Joins

> 스트림과 스트림 데이터집합의 조인의 경우 양쪽 스트림 모두 지연될 수 있기 때문에 완전하지 않은 데이터 소스에 대한 조인이 일어날 수 있으므로 주의해서 다루어야 합니다. 이번에는 click 뿐만 아니라 impression 까지도 스트림 데이터 소스로 조인하는 예제를 학습합니다

![figure.8-11](images/figure.8-11.png)


#### 스트림 간의 조인의 특징

* 엔진은 소스가 스트림 - 스트림 임을 인지하고, impression-and-click 정보의 상태를 버퍼링합니다
* 버퍼링 과정에서 서로 매칭되는 데이터를 통해 조인 연산이 수행됩니다. 코드와 시각화한 화면은 아래와 같습니다.

```python
impressions = spark.readStream.format("kafka").option("...").load()
clicks = spark.readStream.format("kafka").option("...").load()
matched = clicks.join(impressions, "adId")
```

![figure.8-12](images/figure.8-12.png)

> 그림에서의 파란색 점이 impression 과 click 의 '이벤트 타임'을 나타내고 있으며, '마이크로 배치'를 넘나드는 매칭되는 데이터 집합을 통해 조인연산이 일어나게 됩니다.

#### 스트리밍 간의 조인의 가정

* 이러한 '이벤트 타임' 기준 시간으로 조인이 이루어지기 때문에 두 이벤트는 같은 시간 "same wall clock time" 으로 저장되어야 문제가 없습니다
* 위의 이벤트 처리에서는 대기 시간을 지정하지 않았기 때문에 조인되지 않은 데이터의 '스트리밍 상태'는 계속 대기하게 됩니다



#### 스트리밍 간의 조인 처리 시에 아래와 같은 가정을 해보겠습니다

* 광고노출 이후 클릭 사이에 소요되는 최대 시간은 얼마인가?
  - 최대 0초(바로 클릭)에서 최대 한 시간까지 대기했다가 클릭이 발생할 수 있다고 가정해 보았습니다
* 광고의 노출 및 클릭 정보가 네트워크 장애 등의 이슈로 얼마나 지연되어 수신될 수 있는가?
  - 노출 및 클릭이 각 각 2시간, 3시간까지 지연 될 수 있다고 가정해 보았습니다

> 위에서 얘기한 '이벤트 타임' 제약이나 지연 등의 조건은 워터마크와 시간 범위 조건을 이용하여 데이터프레임 연산에 적용할 수 있습니다. 

* 1. 두 가지 '데이터 소스'는 얼마나 지연될 수 있는지 '워터마크'를 통해 정의합니다
* 2. 두 가지 '데이터 소스'간의 '이벤트 타임'에 대한 제약을 정의합니다
  - 오래된 입력 로우가 언제 필요 없어지는 지에 대한 시점을 엔진이 인지하게 되는 제약 조건을 말합니다

* 예제를 통해서 정의해 보겠습니다
  - Time range join conditions : 노출 이후에 클릭이 최대 1시간 정도까지만 발생할 수 있다고 가정했으므로
    * "leftTime BETWEEN rightTime AND rightTime + INTERVAL 1 HOUR"
    * 클릭 시간이 노출 시간 + 1시간 까지의 시간에 대해서 조인될 수 있습니다
  - Join on event-time windows : 
    * leftTimeWindow = rightTimeWindow
    * 클릭의 윈도우 시간은 3시간, 노출은 2시간으로 두었으므로 그 시간 범위에 대해 조인될 수 있습니다

![figure.8-13](images/figure.8-13.png)

#### 그림을 통해 2가지 스트림에 대한 가정을 분석합니다

* 1. 노출 데이터를 기준 : 노출은 2시간 지연이고, 클릭은 3시간 지연이므로, 노출 이후 1시간 클릭 가정을 고려하면 최대 4시간 노출 버퍼가 유지되어야 합니다
* 2. 클릭 데이터를 기준 : 유사하게, 3시간 짜리 클릭 윈도우는 2시간의 버퍼만 유지되더라도 매칭이 가능합니다

```python
impressionsWithWatermark = (
    impressions
    .selectExpr("adId AS impressionAdid", "impressionTime")
    .withWatermark("impressionTime", "2 hours")
)
clickWithWatermark = (
    clicks
    .selectExpr("adId AS clickAdid", "clickTime")
    .withwatermark("clickTime", "3 hours")
)
joined = impressionsWithWatermark.join(
    clickWithWatermark,
    exp("""
        clickAdId = impressionAdid AND
        clickTime BETWEEN impressionTime AND impressionTime + interval 1 hour
        """)
)
```

#### Inner Join 연산에 대한 기억해 두어야하는 키포인트

* Inner Join 의 경우, 워터마킹과, 이벤트 타임 제약 두 가지 모두 선택 사항이므로, 'Unbounded State' 에 대한 위험성을 내포하고 있습니다. 즉, 두 가지 옵션을 지정해 주었을 때에만 더 이상 조인되지 않을 수 있는 '상태'가 정리 대상으로 관리된다는 의미입니다
* 워터마킹 집계 연산에서 다루었던 것과 동일하게, 워터마킹에 의해 지정된 시간 범위내에 지연된 로그가 누락되지 않음을 보장하지만, 지연 로그가 처리되지 않는 것은 보장하지는 않습니다.


### 워터마킹을 이용한 아우터 조인 - Outer joins with watermarking

> 위의 Inner Join 예제의 경우 노출은 되더라도 클릭되지 않은 경우는 전혀 리포팅 되지 않게 됩니다. 그래서 노출은 되지만 클릭되지 않은 경우를 고려하기 위해서 Outer Join 을 사용하게 됩니다

```python
(
    impressionsWithWatermark.join(
        clickWithWatermark,
        expr("""
        clickAdId = impressionAdId AND
        clickTime BETWEEN impressionTime AND impressionTime + interval 1 hour
        """),
        "leftOuter"
    )
)
```

#### Outer Join 의 특징
* 예상대로 매 '마이크로 배치'의 노출에 대한 값에 대해 클릭이 없더라도 출력됩니다
  - Inner Join 의 경우, 워터마킹에 대한 정보가 없고, 제약 시간 조건만 있어도 클릭로그를 처리하지 않는 시점을 인터벌 시간이 지난 클릭로그 시간만으로 인지할 수 있으나,
  - Outer Join 의 경우, '노출 로그'에 매 '마이크로 배치' 시간 마다 NULL 로 채워진 클릭 로그에 대한 결과를 출력해야만 합니다 (지연일 수도, 클릭을 안 한 것일 수도 있으므로), 만약 명시되지 않는 경우 해당 NULL 값을 채워줄 수 없기 때문에 해당 타임 테이블의 출력 시점을 정할 수 없기 때문에 반드시 '워터마킹'과 '인터벌' 정보에 대한 제약을 반드시 명시되어야만 합니다


## 9 임의의 상태 저장 연산 - Arbitrary Stateful Computations

### 9-1. mapGroupsWithState 함수를 이용한 임의의 상태 저장 연산 모델링 - Modeling Arbitrary Stateful Operations with mapGroupsWithState()

### 9-2. 타임아웃을 활용한 비활성 그룹 관리 - Using Timeouts to Manage Inactive Groups

### 9-3. flatMapGroupsWithState 함수를 이용한 일반화 - Generalization with flatMapGroupsWithState()


## 10 성능 개선 - Performance Tuning

> 배치 작업과 다르게 '마이크로 배치' 잡들은 비교적 작은 볼륨의 데이터를 다루기 때문에, 살짝 다른 튜닝 접근을 하고 있습니다.

* 클러스터 리소스 프로비저닝 - Cluster resource provisioning
  - 24/7 서비스를 수행하기 위해 적절한 리소스 프로비저닝이 필수적입니다. 다만, 너무 많은 리소스 할당은 낭비를 초래하고, 적으면 작업이 실패하게 됩니다
  - ***'스테이트리스' 쿼리들은 코어***가 많이 필요지만, ***'스테이트풀' 쿼리들은 상대적으로 메모리***를 많이 필요하기 때문에 쿼리의 특성에 따라 할당하는 것을 고려합니다

* 셔플에 따른 파티션 수 - Number of partitions for shuffles
  - 배치 작업 대비 다소 작은 셔플 파티션의 수를 가지는데, 너무 많은 작업으로 구분하는 것에 따른 오버헤드를 증가시키거나, 처리량을 감소시킬 수 있습니다
  - 또한 셔플링은 '스테이트풀 오퍼레이션'에 따른 '체크포인팅'의 오버헤드가 커질 수 있다는 점도 유의할 필요가 있습니다
  - '스테이트풀' 작업이면서 트리거 간격이 몇 초에서 몇 분인 스트리밍 쿼리의 경우 ***셔플 수를 기본값 인 200에서 할당 된 코어 수의 최대 2~3배로 조정***하는 것이 좋습니다.

* 안정화를 위한 소스 비율 리미트 조정 - Setting source rate limits for stability
  - 초기에 최적화된 설정으로 잘 운영되더라도, 급격하게 늘어난 데이터에 대한 불안정한 상황이 발생할 수 있는데, 리소스를 많이 투입하는 '오버 프로비저닝' 방법 외에도 '소스 속도 제한'을 통한 불안정성을 극복할 수도 있습니다. 
  - 카프카 등의 소스의 임계치를 설정함으로써 싱글 '마이크로 배치' 작업에서 너무 많은 데이터를 처리하지 않도록 막는 방법이며, 증가한 데이터는 소스에서 버퍼링 된 상태로 유지되며, 결국에는 데이터 처리가 따라잡게 됩니다
  - 다만, 아래의 몇 가지를 기억해야만 합니다
    - 리미트를 너무 낮게 설정하면 쿼리가 할당 된 리소스를 충분히 활용하지 못하고 입력 속도보다 느려질 수 있습니다.
    - 리미트 설정 만으로는 입력 속도의 지속적인 증가를 효과적으로 해결하지 못하며, 안정성이 유지되는 동안 버퍼링되고 처리되지 않은 데이터의 양은 소스에서 무한히 증가하므로 종단 간 지연 시간도 증가한다는 점을 알아야 합니다

* 동일한 스파크 애플리케이션 내에서 다수의 스트리밍 쿼리 - Multiple streaming queries in the same Spark application
  - 동일한 SparkContext 또는 SparkSession에서 여러 스트리밍 쿼리를 실행하면 fine-grained 된 리소스 공유가 발생할 수 있습니다.
    - 각 쿼리를 실행하면 Spark 드라이버 (즉, 실행중인 JVM)의 리소스가 계속 사용됩니다. 결국, 드라이버가 동시에 실행할 수있는 쿼리 수가 제한되게 됩니다.
      - 이러한 제한에 도달하면 작업 예약에 병목 현상이 발생하거나 (즉, 실행 프로그램을 제대로 활용하지 못함) 메모리 제한을 초과 할 수 있습니다.
    - 별도의 스케줄러 풀에서 실행되도록 설정하여 동일한 컨텍스트의 쿼리간에보다 공정한 리소스 할당을 보장 할 수 있습니다.
      - SparkContext의 스레드 로컬 속성 spark.scheduler.pool을 각 스트림에 대해 다른 문자열 값으로 설정합니다.

```python
# Run streaming query1 in scheduler pool1
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool1")
df.writeStream.queryName("query1").format("parquet").start(path1)

# Run streaming query2 in scheduler pool2
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool2")
df.writeStream.queryName("query2").format("parquet").start(path2)
```

In [7]:
# 스트리밍 데이터에 대한 디버깅을 위해 사전 정의한 함수
def foreach_batch_function(df, epoch_id):
    print("{} - {}".format(epoch_id, df.collect()))

streamingQuery = (
    counts
    .writeStream
    .format("memory")
    .outputMode("complete")
    .trigger(processingTime="1 second") # 1 second micro batch interval
    .foreachBatch(foreach_batch_function)
    .start()
)
streamingQuery.awaitTermination()

StreamingQueryException: Connection refused (Connection refused)
=== Streaming Query ===
Identifier: [id = cf94b137-8b9d-4198-826b-bfd6909f87df, runId = e8673ef6-a4fb-491d-b478-dc530c53f34f]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 9999]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
SubqueryAlias count
+- Aggregate [word#17], [word#17, count(1) AS count#21L]
   +- Project [split(value#15, \s, -1) AS word#17]
      +- StreamingDataSourceV2Relation [value#15], org.apache.spark.sql.execution.streaming.sources.TextSocketTable$$anon$1@2cc0d0a4, TextSocketV2[host: localhost, port: 9999]


In [24]:
!rm -rf "tmp/checkpoint"

import calendar;
import time;
ts = calendar.timegm(time.gmtime())
print(ts)
# 1624239264

1624735618


In [5]:
from pyspark.sql.functions import *

checkpointDir = "tmp/checkpoint"
words = lines.select(split(col("value"), "\\s").alias("word"))
counts = words.groupBy("word").count()
streamingQuery = (
    counts
    .writeStream
    .format("console")
    .outputMode("complete")
    .trigger(processingTime="1 second") # 1 second micro batch interval
    .option("checkpointLocation", checkpointDir)
    .start()
)
streamingQuery.awaitTermination()

KeyboardInterrupt: 

In [37]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at <ipython-input-1-a6d1a7399678>:4 

In [2]:
# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

In [3]:
# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

In [4]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [5]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2021-05-11 15:14:33
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:34
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:35
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:36
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:37
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:38
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:39
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:40
-------------------------------------------
('test', 1)

-------------------------------------------
Time: 2021-05-11 15:14:4

KeyboardInterrupt: 

-------------------------------------------
Time: 2021-05-11 15:14:51
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:52
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:53
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:54
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:55
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:56
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:57
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:58
-------------------------------------------

-------------------------------------------
Time: 2021-05-11 15:14:59
----------

### References
* [Streaming Programming Guide](https://spark.apache.org/docs/latest/streaming-programming-guide.html)
* [Spark Runtime Configuration Guide](https://spark.apache.org/docs/latest/configuration.html#spark-sql)
* 5-2. [Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#creating-streaming-dataframes-and-streaming-datasets)
* 5-2. [Apache Kafka Client](https://docs.confluent.io/clients-confluent-kafka-python/current/overview.html)
  - [Kafka python on github](https://github.com/dpkp/kafka-python/blob/master/example.py)
  - [Kafka Producer & Consumer](https://needjarvis.tistory.com/607)
* 5-2. [Cassandar Data Manipulation](https://cassandra.apache.org/doc/latest/cql/dml.html)
  - [Creating Cassandra keyspace & table on docker start up](https://www.linkedin.com/pulse/creating-cassandra-keyspace-table-docker-start-up-amon-peter/)
* 5-2. [MySQL JDBC Connection Example](https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html)
* [Linux Kill](https://www.lesstif.com/system-admin/unix-linux-kill-12943674.html)
* [Python Current Timestamp](https://timestamp.online/article/how-to-get-current-timestamp-in-python)
* [NetCat on Windows](https://mentha2.tistory.com/65)
* [NetCat on Ubuntu](https://epicarts.tistory.com/43)
* [VisualVM](https://visualvm.github.io/download.html)
* [Monitoring spark w/ prometheus](https://argus-sec.com/monitoring-spark-prometheus/)
* [Dynamic Docker Monitoriong](https://www.datadoghq.com/dg/monitor/docker-benefits/)

## 11. 자주 사용되는 용어
* Structured API 란?
  - 일반적인 함수 혹은 메소드 형식의 API 와 다르게, 데이터프레임을 통하여 다양한 메소드를 연결하여 활용할 수 있는 구조를 가졌기 때문에 구조화된 API 라고 말할 수 있습니다
* Map-Reduce 란?
  - 병렬처리가 가능한 상태가 없는 단순 계산 연산을 Map, 병렬처리가 불가능한 상태가 존재하는 집계 연산을 Reduce 라고말할 수 있으며, 가장 쉬운 예제로 WordCount 가 있습니다
* Idempotence 란?
  - 반복적으로 수행 혹은 처리하여도 항상 결과가 동일하게 나오는 것
* Structured Streaming vs. DStreams ?
  - DStream : RDD 를 이용하여 스트리밍 처리를 직접 구현할 수 있는 Spark 1.x 버전부터 제공 되었던 인터페이스를 말합니다
  - Structured Streaming : Spark 2.x 버전부터 제공 되는 Spark SQL 즉, Dataframe (Dataset) 기반의 API 통한 스트리밍 처리 인터페이스를 말합니다
* [RDD vs. Dataset vs. Dataframe](https://www.analyticsvidhya.com/blog/2020/11/what-is-the-difference-between-rdds-dataframes-and-datasets/)
  - RDD : 최적화 및 데이터 처리에 대한 구현을 직접해야 하는 경우
```python
sc = SparkContext("local","PySpark Word Count Exmaple")
words = sc.textFile("tmp/source").flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
wordCounts.saveAsTextFile("tmp/target")
```
  - Dataframe : 제공되는 API 통한 데이터 처리를 수행하는 경우
```python
df = spark.read.text("tmp/source")
wordCounts = df.withColumn("word", explode(split("value", " "))).groupBy("word").count()
wordCounts.write.format("text").save("tmp/target")
```
  - Dataset : Typed Compile 언어인 Java, Scala 만 지원하며, Compile 시점에 Type-safe 보장
* Event-time vs. Processing-time ?
  - Event-time : 실제 사건 혹은 이벤트가 발생한 시간 (ex_ 핸드폰에서 광고를 클릭한 시간)
  - Processing-time : 데이터를 수집 혹은 수신한 시간 (ex_ 브로커를 통해 로그가 수신된 시간)
* Bounded vs. Unbounded ?
  - Bounded : 데이터를 처리하는 시점에 그 범위가 명확하게 정해진 경우를 말합니다 (ex_ 배치 처리와 같이 어제 하루에 수신된 모든 로그를 처리해야 하는 경우)
  - Unbounded : 데이터를 처리하는 시점에 그 범위가 명확하게 정해지지 않은 경우를 말합니다 (ex_ 스트리밍 데이터와 같이 끝없이 이어지는 데이터를 수신하고 처리하는 경우)
* Source vs. Sink ?
  - Source : 처리해야 하는 데이터 소스 위치
  - Sink : 데이터를 저장 혹은 전달하는 타겟 위치
* incrementalization ?
  - 스트리밍 처리에서 스파크는 Unbounded 데이터를 마치 정적인 테이블인 것처럼 동작하게 하고, 최종 출력 싱크에 저장될 결과 테이블을 계산합니다.
  - **여기서의 배치와 유사한 쿼리 수행을 스트리밍 실행 계획으로 변환**하게 하는데 이 과정을 '증분화(*incrementalization*)'라고 합니다. 
  - 스파크는 레코드가 도착할 때마다 결과를 업데이트 하기 위해 어떤 상태를 유지해야 하는지를 확인하여 점진적으로 결과를 업데이트 합니다.
  - 즉, 마치 배치 처리를 통한 소스 테이블 쿼리 후 타겟 테이블에 저장하는 것 처럼 보이지만, 실제로는 증분화된 스트리밍 처리 계획이며 이를 증분화라고 합니다
* materializing ?
  - RDD 혹은 Dataframe 의 데이터 객체 상태의 meterialized 의 의미는 lazy evaluation 관점에서 보았을 때에 not materialized 의 의미는 실행 계획이 아직 검토되지 않았다고 말할 수 있으며, materialized 되었다는 의미는 실행계획을 통해 대상 데이터가 메모리에 올라와서 접근가능한 상태정도라고 말할 수 있습니다.
    - we allocate memory for it. That is, we store it into memory (.persist()) or even store it into durable storage (.persist(RELIABLE)).
    - Sparks runtime does lazy evaluation so until an action is taken the RDD is not materialized.
  - 한편으로 논리적인 테이블 관점에서 보았을 때에는, createOrReplaceTempView 는 가상의 테이블에 대한 메타정보만 가진 테이블을 말하며, saveAsTable 명령은 실제 데이터 프레임을 물리적인 저장 경로에 저장한 상태를 말합니다.
    - Unlike the createOrReplaceTempView command, saveAsTable will materialize the contents of the DataFrame and create a pointer to the data in the Hive metastore. 
  - 참고로 스파크에서는 meterialized view 를 아직 지원하지(SPARK-29038) 않습니다
* epoch ?
  - 컴퓨팅 컨텍스트에서 에포크는 컴퓨터의 시계 및 타임스탬프 값이 결정되는 날짜와 시간입니다.
  - 에포크는 일반적으로 시스템마다 다른 특정 날짜의 0시 0분 0초(00:00:00) UTC(협정 세계시)에 해당합니다.
  - 예를 들어 대부분의 Unix 버전은 1970년 1월 1일을 epoch 날짜로 사용합니다. - [Epoch Time](https://searchdatacenter.techtarget.com/definition/epoch)
    - 스트리밍 처리에 있어서 데이터 처리 트랜잭션 관리 및 유지를 위해 commit log 의 commits 정보를 metadata checkpoint directory 에 저장하게 됩니다.
    - `Offset Commit Log — commits Metadata Checkpoint Directory` 체크포인트 경로에 메타데이터를 커밋하는 행위를 말합니다.
* micro-batch ?
  - 스파크의 Structured Streaming 은 엄밀히 말하면 Continuous Streaming 처리가 아니라 500ms 수준의 작은 배치 작업으로 쪼개어 (deterministic) 수행하는데 이 작은 배치 작업을 말합니다
    - Minimum batch size Spark Streaming can use.is 500 milliseconds, is has proven to be a good minimum size for many applications.
    - The best approach is to start with a larger batch size (around 10 seconds) and work your way down to a smaller batch size.
* *stateless* and *stateful* in spark transformation?
  - stateless : 스트리밍 처리에 있어 이전 상태에 의존하지 않고 현재의 상태만 이용하여 변환 혹은 처리를 수행하는 경우 (ex_ 현재 그룹 내의 빈도 혹은 특정 값의 상태)
  - stateful : 스트리밍 처리에 있어 이전 상태에 의존한 데이터 변환을 말합니다 (ex_ 누적 값 혹은 집계 연산 결과 상태)


## 12. 예상되는 질문들

* 스트리밍의 경우 explain 명령을 통해 '로지컬 플랜' 혹은 '최적화된 플랜'을 확인할 수 없나요?
  - 스트리밍 쿼리에 대해 아래와 같이 explain 함수를 통해 확인할 수 있습니다
```python
query.processAllAvailable()
query.explain()
```

* 동일한 키가 존재하는 정렬의 경우 왜 결정론적이지 못 한가요?
  - 분산환경에서 셔플링이 발생하는 경우 일시적인 네트워크 지연, 특정 노드의 장애에 따라 Reduce 노드에 도착하는 데이터의 순서는 언제든지 변경될 수 있습니다

* 스트리밍 처리가 1초 단위라고 했는데 소캣을 통한 테스트 시에는 화면 출력이 아주 느려 보이는데 왜 그런가요?
  - 내부 트리거는 계속 발생하지만, 데이터 소스로부터 가져올 데이터가 없기 때문에 수행되지 않는 것처럼 보입니다
  - 화면에 출력되는 것은 최종 출력 싱크에 트리거로부터 저장할 데이터가 발생하는 경우에만 발생하는 이벤트라서 그렇습니다

* 계속 지켜보면 데이터가 없지만 결과 테이블이 출력되는 경우도 있는데 왜 그런가요?
  - 메시지를 잘 살펴보시면 아래와 같은데요, 해석해 보면, '현재 배치는 지연되고 있으며, 트리거는 1초 이지만 약 8초 정도 소요되었다'라고 나오고 있습니다.
  - `21/06/27 08:19:06 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 8314 milliseconds`
  - '스파크 스트리밍' 처리에 대한 기준은 몇 가지가 있는데 1: 초기 기동 시에 실행, 2: 소스에 처리할 데이터가 존재하는 경우 실행, 3: 임계시간이 지난경우 

* JSON 혹은 header 가 있는 TSV 등은 스키마 infer 가 가능한데 왜 schema 를 명시적으로 생성해 주어야 하나요?
  - '스파크 스트리밍'의 경우 데이터 타입에 민감하며 자칫 실수하는 경우 전체 스트리밍 파이프라인의 장애로 이어지기 때문에 엄격한 스키마 정의가 반드시 필요합니다
  - spark.readStream.schema(schema) 와 같이 정의되어야만 하며, 그렇지 않으면 아래와 같은 오류를 출력합니다
  - `IllegalArgumentException: Schema must be specified when creating a streaming source DataFrame. If some files already exist in the directory, then depending on the file format you may be able to create a static DataFrame on that directory with 'spark.read.load(directory)' and infer schema from it.`

* 집계함수 출력이 안 되는데 왜 그런가?
  - 기본 출력 모드가 append 모드이기 떄문에, watermark 지정을 해주지 않는 경우 집계함수를 append 모드 통해서 저장할 수 없습니다. (consistency 문제)
  - 출력모드를 console complete 로 지정하여 테스트하거나, watermark 를 통해 출력하여야만 하며 그렇지 않으면 아래와 같은 오류를 출력합니다
  - `AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark`

* 입력 데이터 지정시에 JSON 파일을 넣을 수는 없는가?
  - '스파크 스트리밍' 데이터는 파일 단위로 생성된 데이터 여부를 판단하기 때문에 경로를 입력해야만 합니다 그렇지 않으면 아래와 같은 오류를 콘솔에 출력합니다
  - `java.lang.IllegalArgumentException: Option 'basePath' must be a directory`
    
* 카프카 엔진의 포트가 왜 9092, 9093 으로 나누어져 있고, kafka:9093, localhost:9092 로 접근해야만 하는가?
  - 카프카 디폴트 포트는 9092 포트입니다
  - 리스너 설정 시에 기본 구성의 경우 `listeners=PLAINTEXT://broker1:9091,SSL://broker1:9092,SASL_SSL://broker1:9093` 으로 9091 ~ 9093 까지 용도에 따라 설정합니다
  - [Security Protocol](http://kafka.apache.org/11/javadoc/org/apache/kafka/common/security/auth/SecurityProtocol.html) - 프로토콜은 아래와 같습니다.
  | Enum Constant | Description |
  | --- | --- |
  | PLAINTEXT | Un-authenticated, non-encrypted channel |
  | SASL_PLAINTEXT | SASL authenticated, non-encrypted channel |
  | SASL_SSL | SASL authenticated, SSL channel |
  | SSL | SSL channel |

* 스파크 스트리밍도 배치 처리와 유사하게 몇 개의 익스큐터가 동시에 기동되고 항상 떠서 서로간의 데이터를 주고 받는 방식인가요?
  - 익스큐터들 내에 집계 연산을 위해 셔플을 통해 데이터를 전송합니다

* 텀블링 윈도우는 항상 그 시간대를 유지하면서 메모리에 저장되어 있는가?
  - 워터마크가 없는 스트리밍은 끝없이 출력되는 것으로 간주하고 수행됩니다



> 중요한 질문

* 스트리밍 윈도우 워터마크 동작방식이 이벤트타임 시간을 기준으로 상대적인 시간인지, 마지막 시간 이후의 절대적 시간인지?

* 수행할 수 있는 Executor 수를 임의로 지정할 수 있는지?

* dynamic allocation 방식이 streaming 은 더 유용한 것이 아닌지?

* 상태를 backend storage 에 저장하게 되는지 여기는 어디인지? hdfs? local disk?

* 파일을 통한 스트리밍 읽을 때에는 추가된 내용에 대해서는 왜 처리가 안되는지, 그리고 한 번에 모든 데이터가 처리되는지?

* 지연된다고 하더라도 집계에 포함되는 경우도 있다고 하는데 언제 이러한 현상이 발생하는가? 위에서 만든 생성자 예제와 비슷한 상황인가?

> 기타 질문

* 스트리밍 처리를 통한 화면 출력이 즉각적으로 출력되지 않고 몇 초 간의 딜레이가 있는데 왜 그런지, 어떻게 바로 출력할 수 있는지?

* 스트리밍 모니터링을 위해서 awaitTermination 호출이 진행되고 있는 동안에는 상태를 확인할 수 없는데 어떻게 모니터링하면 좋은가?

* 스트리밍 모니터링을 JMX 통한 매트릭 조회는 불가능한가요?

* 입력은 TSV 가 되는데 출력은 왜 TSV 로 할 수 없는가?

* 카프카 연동 시에 다양한 옵션에 대한 설명과 예제는 없나요?

* 옵션을 주지 않으면 텀블링 윈도우인가? 구별은 어떻게 하는가

* 스트리밍 결과가 지연된 데이터 출력 때문에 시간 순으로 보이지 않는데 어떻게 정렬할 수 있나요?

* 스트리밍 조인 시에 '스트리밍 쿼리'를 재시작 하지 않고 반영할 수 있는 정적 데이터 소스에는 어떤 것들이 있나요?

* 하나의 스파크 컨텍스트에서 여러개의 쿼리를 어떻게 수행할 수 있나요?


In [1]:
from pyspark.sql import *

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)